In [1]:
import tensorflow as tf
from tensorflow import keras
import os

In [2]:
!unrar x -Y drive/MyDrive/dataset.rar


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from drive/MyDrive/dataset.rar

Creating    dataset                                                   OK
Creating    dataset/test                                              OK
Creating    dataset/test/Elephant                                     OK
Extracting  dataset/test/Elephant/028f8db193a85ecf.jpg                     0%  OK 
Extracting  dataset/test/Elephant/0ba8f4463ed0513f.jpg                     0%  OK 
Extracting  dataset/test/Elephant/140a529acb32f8d1.jpg                     0%  OK 
Extracting  dataset/test/Elephant/1a676e236a83f4ec.jpg                     0%  OK 
Extracting  dataset/test/Elephant/1de68e44f26155a7.jpg                     0%  OK 
Extracting  dataset/test/Elephant/226693882e6bfe60.jpg                     0%  OK 
Extracting  dataset/test/Elephant/2a9394aafbe6e9fa.jpg                     0%  OK 
Extracting  dataset/test/

In [3]:
base_dir = 'dataset/train'

In [4]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size = BATCH_SIZE,
    subset='training'
)
val_generator = datagen.flow_from_directory(  #validation generator
    base_dir, 
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'
)

Found 829 images belonging to 5 classes.
Found 205 images belonging to 5 classes.


In [5]:
print(train_generator.class_indices)
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)

{'Elephant': 0, 'Kangaroo': 1, 'Panda': 2, 'Penguin': 3, 'Tiger': 4}


In [6]:
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3) 
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

9420800/9406464 [==============================] - 0s 0us/step


In [7]:
base_model.trainable=False
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.Conv2D(32,3, activation = 'relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.Dense(5, #no.of classes
                        activation='softmax')
])

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [9]:
epochs = 10
history = model.fit(
    train_generator, 
    epochs = epochs, 
    validation_data=val_generator
)

Epoch 1/10
13/13 [==============================] - 50s 1s/step - loss: 1.4504 - accuracy: 0.6502 - val_loss: 0.2795 - val_accuracy: 0.9268
Epoch 2/10
13/13 [==============================] - 17s 1s/step - loss: 0.1619 - accuracy: 0.9505 - val_loss: 0.2219 - val_accuracy: 0.9366
Epoch 3/10
13/13 [==============================] - 16s 1s/step - loss: 0.0669 - accuracy: 0.9783 - val_loss: 0.1283 - val_accuracy: 0.9561
Epoch 4/10
13/13 [==============================] - 16s 1s/step - loss: 0.0268 - accuracy: 0.9928 - val_loss: 0.1174 - val_accuracy: 0.9707
Epoch 5/10
13/13 [==============================] - 16s 1s/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 0.1209 - val_accuracy: 0.9659
Epoch 6/10
13/13 [==============================] - 16s 1s/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 0.1336 - val_accuracy: 0.9659
Epoch 7/10
13/13 [==============================] - 16s 1s/step - loss: 0.0034 - accuracy: 1.0000 - val_loss: 0.1264 - val_accuracy: 0.9610
Epoch 8/10
13/13 [==

In [10]:
saved_model_dir = ''
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: assets


INFO:tensorflow:Assets written to: assets


In [11]:
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>